We built a model in 3.0-theberling-classifier.ipynb but we didn't have the right submissions. So now we're going to reverse engineer from their sample submission file to get the year, and team IDs that we need. Let's start by reading in their sample submission data file.

In [2]:
import pandas as pd

In [13]:
df_submission_sample = pd.read_csv('../../data/raw/MSampleSubmissionStage1.csv')

In [14]:
df_submission_sample.head()

,ID,Pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5


In [15]:
new = df_submission_sample["ID"].str.split("_", expand = True)
new

,0,1,2
0,2015,1107,1112
1,2015,1107,1116
2,2015,1107,1124
3,2015,1107,1125
4,2015,1107,1129
...,...,...,...
11385,2019,1449,1459
11386,2019,1449,1463
11387,2019,1458,1459
11388,2019,1458,1463


In [23]:
combinations = pd.DataFrame()
combinations['Season'] = new[0]
combinations['LowID'] = new[1]
combinations['HighID'] = new[2]
combinations['Season'] = pd.to_numeric(combinations['Season'])
combinations['LowID'] = pd.to_numeric(combinations['LowID'])
combinations['HighID'] = pd.to_numeric(combinations['HighID'])
combinations.head()

,Season,LowID,HighID
0,2015,1107,1112
1,2015,1107,1116
2,2015,1107,1124
3,2015,1107,1125
4,2015,1107,1129


Now let's load up the season averages data.

In [24]:
df = pd.read_csv('../../data/processed/data_averages.csv')

Let's merge the low ID team with it's data by season. To do that we'll temporarily rename LowID to TeamID and merge on Season and TeamID.

In [25]:
LowID = pd.merge(combinations.rename({'LowID': 'TeamID'}, axis=1), df, how="left", on=['Season','TeamID']).dropna().reset_index(drop=True)
LowID.head()

,Season,TeamID,HighID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed
0,2015,1107,1112,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0
1,2015,1107,1116,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0
2,2015,1107,1124,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0
3,2015,1107,1125,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0
4,2015,1107,1129,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0


In [28]:
LowID.columns = ['Season','LowID','HighID','LowScore','LowFGP','LowFGP3','LowFTP','LowOR','LowDR','LowAst','LowTO','LowStl','LowBlk','LowPF','LowRank','LowSeed']
LowID.head()

,Season,LowID,HighID,LowScore,LowFGP,LowFGP3,LowFTP,LowOR,LowDR,LowAst,LowTO,LowStl,LowBlk,LowPF,LowRank,LowSeed
0,2015,1107,1112,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0
1,2015,1107,1116,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0
2,2015,1107,1124,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0
3,2015,1107,1125,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0
4,2015,1107,1129,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,11.6875,5.65625,1.625,16.5,118.87931,14.0


In [29]:
HighID = pd.merge(combinations.rename({'HighID': 'TeamID'}, axis=1), df, how="left", on=['Season','TeamID']).dropna().reset_index(drop=True)
HighID.head()

,Season,LowID,TeamID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed
0,2015,1107,1112,76.441176,0.486694,0.358874,0.699585,10.823529,26.411765,14.205882,11.205882,7.176471,3.588235,17.911765,4.209677,2.0
1,2015,1107,1116,78.029412,0.448082,0.347178,0.717672,13.000000,22.764706,16.147059,11.735294,7.764706,4.764706,18.970588,22.306452,5.0
2,2015,1107,1124,69.125000,0.435875,0.379881,0.668958,14.500000,24.031250,14.406250,12.375000,8.031250,3.843750,16.718750,14.177419,3.0
3,2015,1107,1125,73.774194,0.474480,0.378229,0.695699,9.322581,23.967742,15.193548,13.741935,6.354839,1.967742,16.451613,129.724138,15.0
4,2015,1107,1129,69.870968,0.455117,0.394584,0.729914,8.516129,24.032258,11.709677,10.225806,5.967742,2.322581,17.225806,46.700000,11.0


In [31]:
HighID.columns = ['Season','LowID','HighID','HighScore','HighFGP','HighFGP3','HighFTP','HighOR','HighDR','HighAst','HighTO','HighStl','HighBlk','HighPF','HighRank','HighSeed']
HighID.head()

,Season,LowID,HighID,HighScore,HighFGP,HighFGP3,HighFTP,HighOR,HighDR,HighAst,HighTO,HighStl,HighBlk,HighPF,HighRank,HighSeed
0,2015,1107,1112,76.441176,0.486694,0.358874,0.699585,10.823529,26.411765,14.205882,11.205882,7.176471,3.588235,17.911765,4.209677,2.0
1,2015,1107,1116,78.029412,0.448082,0.347178,0.717672,13.000000,22.764706,16.147059,11.735294,7.764706,4.764706,18.970588,22.306452,5.0
2,2015,1107,1124,69.125000,0.435875,0.379881,0.668958,14.500000,24.031250,14.406250,12.375000,8.031250,3.843750,16.718750,14.177419,3.0
3,2015,1107,1125,73.774194,0.474480,0.378229,0.695699,9.322581,23.967742,15.193548,13.741935,6.354839,1.967742,16.451613,129.724138,15.0
4,2015,1107,1129,69.870968,0.455117,0.394584,0.729914,8.516129,24.032258,11.709677,10.225806,5.967742,2.322581,17.225806,46.700000,11.0


And finally we can merge the HighID and LowID DataFrame on Season, LowID, HighID, and Win since those should be the same in each DataFrame.

In [33]:
model_dataset = pd.merge(LowID, HighID, how='outer', on=['Season','LowID','HighID'])
model_dataset.head()

,Season,LowID,HighID,LowScore,LowFGP,LowFGP3,LowFTP,LowOR,LowDR,LowAst,...,HighFTP,HighOR,HighDR,HighAst,HighTO,HighStl,HighBlk,HighPF,HighRank,HighSeed
0,2015,1107,1112,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,...,0.699585,10.823529,26.411765,14.205882,11.205882,7.176471,3.588235,17.911765,4.209677,2.0
1,2015,1107,1116,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,...,0.717672,13.000000,22.764706,16.147059,11.735294,7.764706,4.764706,18.970588,22.306452,5.0
2,2015,1107,1124,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,...,0.668958,14.500000,24.031250,14.406250,12.375000,8.031250,3.843750,16.718750,14.177419,3.0
3,2015,1107,1125,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,...,0.695699,9.322581,23.967742,15.193548,13.741935,6.354839,1.967742,16.451613,129.724138,15.0
4,2015,1107,1129,65.5,0.44403,0.358773,0.728865,10.0,23.625,10.4375,...,0.729914,8.516129,24.032258,11.709677,10.225806,5.967742,2.322581,17.225806,46.700000,11.0


In [34]:
model_dataset.shape

(11390, 29)

In [36]:
model_dataset.to_csv('../../data/processed/model_dataset2.csv',index=False)